In [1]:
%matplotlib ipympl
import numpy as np
from mask_utils import *
import matplotlib.pyplot as plt
from matplotlib import cm

In [20]:
def fits_mask_to_dxf(fitsin, dxfout):
    def get_label_index(arr, _label):
        # Returns the first and last indexes of a labeled region
        # e.g. 
        # (x1, y1), (x2, y2) = get_label_index(l, 2)
        idx = np.where(arr == _label)
        return (idx[0][0], idx[1][0]), (idx[0][-1], idx[1][-1])

    #reading mask file as an image
    mask, header = read_mask_bulk(fitsin, 'MASK', header_out=True, verbose=False)
    ELXDIM = header['ELXDIM']
    ELYDIM = header['ELYDIM']
    MXDIM  = header['MXDIM']
    MYDIM  = header['MYDIM']

    #Finding open regions
    l, num_features = label(mask)

    #Initializing DXF file
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()

    for feature in range(1, num_features):
        #Getting open region corner indexes
        (i1, j1), (i2, j2) = get_label_index(l, feature)
        #Transforming indexes in spatial coordinates
        x1, y1 = i1 * ELXDIM -  MXDIM/2,  j1 * ELYDIM -  MYDIM/2
        x2, y2 = (i2+1) * ELXDIM -  MXDIM/2, (j2+1) * ELYDIM - MYDIM/2

        #Drawing rectangle as closed polyline
        msp.add_lwpolyline([(x1, y1),(x2, y1),(x2, y2),(x1, y2),(x1, y1)],close=True)

    #Writing DXF file
    doc.saveas(dxfout)
    print(dxfout, 'file created')

In [21]:
fits_mask_to_dxf('mask_050_1040x17_20250709.fits', 'prova2.dxf')

prova2.dxf file created
